In [607]:
import sys
sys.path.append("../")
from lib.clean import standardize_desc_cols, clean_column_names
from lib.uid import gen_uid

In [608]:
import pandas as pd
from datamatch import (
    JaroWinklerSimilarity,
    ThresholdMatcher,
    ColumnsIndex,
)

In [609]:
def read_data():
    df = pd.read_csv("../data/OK/aggregate_members_oathkeepers_cleaned.csv")
    return df 

In [610]:
dfa = read_data()

In [611]:
dfa = dfa.fillna("")

In [612]:
dfa.loc[:, "state"] = dfa.state.str.lower().str.strip().str.replace(r"ma\.", "ma", regex=True)

In [613]:
dfa.columns

Index(['id_number', 'group_name', 'bool', 'city', 'state', 'email',
       'undefined_date', 'notes', 'first_name', 'middle_name', 'last_name'],
      dtype='object')

In [614]:
dfa = dfa[dfa.state.isin(["ma"])]
dfa.loc[:, "notes"] = dfa.notes.str.lower().str.strip()
dfa.loc[:, "first_name"] = dfa.first_name.str.replace(r"^ (\w+)", r"\1", regex=True).str.replace(r"(\w+) $", r"\1", regex=True)
dfa.loc[:, "last_name"] = dfa.last_name.str.replace(r"^ (\w+)", r"\1", regex=True).str.replace(r"(\w+) $", r"\1", regex=True)
dfa.loc[:, "city"] = dfa.city.str.lower().str.strip().str.replace(r"\,.+", "", regex=True)

In [615]:
dfa.state.unique()

array(['ma'], dtype=object)

In [ ]:
search_1 = dfa[dfa.notes.str.contains("law enforcement")]
search_1

In [ ]:
search_2 = dfa[dfa.notes.str.contains("police")]
search_2

In [ ]:
search_3 = dfa[dfa.notes.str.contains("patrol")]
search_3.notes.unique()

In [ ]:
search_4 = dfa[dfa.notes.str.contains("officer")]
search_4.notes.unique()

In [ ]:
search_5 = dfa[dfa.notes.str.contains("swat")]
search_5

In [ ]:
search_6 = dfa[dfa.notes.str.contains("explosive")]
search_6

In [ ]:
search_7 = dfa[dfa.notes.str.contains("bomb")]
search_7

In [ ]:
search_8 = dfa[dfa.notes.str.contains("active duty")]
search_8.notes.unique()

In [624]:
dfs = [search_1, search_2, search_3, search_4, search_5, search_6, search_7, search_8]
dfb = pd.concat(dfs, axis=0)

In [625]:
def read_misconduct_data():
    df1 = pd.read_csv("../data/MA/misconduct/nacdl/billerica/ia/uof_billerica_2015_2020.csv")
    df1["agency"] = "billerica"
    df2 = pd.read_csv("../data/MA/misconduct/nacdl/Brookline/IIU/cprr_brookline_2010_2021.csv")
    df2["agency"] = "brookline"
    df3 = pd.read_csv("../data/MA/misconduct/nacdl/Chelmsford/IA/cprr_chelmsford_2015_2016.csv")
    df3["agency"] = "chelmsford"
    df4 = pd.read_csv("../data/MA/misconduct/nacdl/Dartmouth/IIU/cprr_dartmouth_2013_2022.csv")
    df4["agency"] = "dartmouth"
    df5 = pd.read_csv("../data/MA/misconduct/nacdl/Framingham/IIU/cprr_framingham_2011_2021.csv")
    df5["agency"] = "framingham"
    df6 = pd.read_csv("../data/MA/misconduct/nacdl/Milton/IA/cprr_milton_2010_2019.csv")
    df6["agency"] = "milton"
    df7 = pd.read_csv("../data/MA/misconduct/nacdl/Newton/IIU/cprr_newton_2010_2021.csv")
    df7["agency"] = "newton"
    df8 = pd.read_csv("../data/MA/misconduct/nacdl/Norwood/IIU/cprr_norwood_2011_2020.csv")
    df8["agency"] = "norwood"
    df9 = pd.read_csv("../data/MA/misconduct/nacdl/Quincy/IIU/cprr_quincy_2018_2020.csv")
    df9["agency"] = "quincy"
    df10 = pd.read_csv("../data/MA/misconduct/nacdl/Wellesley/IIU/cprr_wellesley_2010_2020.csv")
    df10["agency"] = "wellesley"
    df11 = pd.read_csv("../data/MA/misconduct/nacdl/Westfield/IIU/cprr_westfield_2010_2019.csv")
    df11["agency"] = "westfield"
    df12 = pd.read_csv("../data/MA/misconduct/nacdl/Weymouth/IIU/cprr_weymouth_2009_2020.csv")
    df12["agency"] = "weymouth"
    df13 = pd.read_csv("../data/MA/misconduct/nacdl/Woburn/IIU/cprr_woburn_2011_2021.csv")
    df13["agency"] = "woburn"
    return df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13

In [626]:
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13 = read_misconduct_data()

In [627]:
def split_names(df):
    df = df.pipe(clean_column_names)
    df = df[~((df.name.fillna("") == ""))]

    names = df.name.str.lower().str.strip().str.extract(r"(.+)\,? (?:(\w+\'?\w+?)) ?(\w+)?")
    
    df.loc[:, "last_name"] = names[0].str.replace(r"\,", "", regex=True)
    df.loc[:, "first_name"] = names[1]
    df.loc[:, "middle_name"] = names[2]
    return df[~((df.first_name.fillna("") == "") & (df.last_name.fillna("") == ""))]

def split_names_f_pf(df):
    df = df.pipe(clean_column_names)
    df = df[~((df.name.fillna("") == ""))]

    names = df.name.str.lower().str.strip().str.extract(r"(.+)\.? (?:(\w+\'?\w+?))")
    
    df.loc[:, "first_name"] = names[0].str.replace(r"\.", "", regex=True)
    
    df.loc[:, "last_name"] = names[1].str.replace(r"\,", "", regex=True)
    return df[~((df.first_name.fillna("") == "") & (df.last_name.fillna("") == ""))]


def split_names_v3(df):
    df = df[~((df.name.fillna("") == ""))]

    names = df.name.str.lower().str.strip().str.extract(r"(?:(\w+)\,? ?)(\w+)?")
    
    df.loc[:, "last_name"] = names[0]
    df.loc[:, "first_name"] = names[1]
    return df[~((df.first_name.fillna("") == "") & (df.last_name.fillna("") == ""))]

def split_names_v4(df):
    df = df[~((df.name.fillna("") == ""))]

    names = df.name.str.lower().str.strip().str.extract(r"(?:(\w+)\,? ?)(\w+)?")
    
    df.loc[:, "first_name"] = names[0]
    df.loc[:, "last_name"] = names[1]
    return df[~((df.first_name.fillna("") == "") & (df.last_name.fillna("") == ""))]

def strip_titles(df):
    df.loc[:, "name"] = df.name.str.lower().str.strip().replace(r"lt\.?\b", "", regex=True).str.replace(r"sgt ?", "", regex=True)\
        .str.replace(r"captain ?", "", regex=True).str.replace(r"officers? ?", "", regex=True).str.replace(r"sergeant ?", "", regex=True)
    return df 

def strip_leading_aps(df):
    df = df.apply(lambda x: x.str.replace(r"^\'", "", regex=True))
    return df 

In [628]:
def split_rows_with_multiple_names(df):
    df.loc[:, "name"] = (df.name
        .str.replace(r"(\w+) \/ (\w+)", r"\1/\2", regex=True)
        .str.replace(r"^\'", "", regex=True)
        .str.replace(r"(\w+ \w+) (\w+ \w+)", r"\1/\2", regex=True)
    )
    df = (
        df.drop("name", axis=1)
        .join(
            df["name"]
            .str.split("/", expand=True)
            .stack()
            .reset_index(level=1, drop=True)
            .rename("name"),
            how="outer",
        )
        .reset_index(drop=True)
    )
    return df

In [ ]:
df1[["first_name", "middle_name"]] = ""

df2 = df2.pipe(split_names)

df3 = df3.pipe(clean_column_names)
df3 = df3.rename(columns={"name": "last_name"})
df3[["first_name", "middle_name"]] = ""

df4 = df4.pipe(split_names)

df5 = df5.pipe(split_rows_with_multiple_names).pipe(split_names)

df6 = (df6
        .pipe(clean_column_names)
        .rename(columns={"officer": "name"})
        .pipe(split_rows_with_multiple_names)
        .pipe(split_names_f_pf)
        )

df6[["middle_name"]] = ""

df7[["middle_name"]] = ""

df8_a = (df8.pipe(clean_column_names).rename(columns={"involved_officer": "name"}).pipe(split_rows_with_multiple_names).pipe(split_names_v3))
df8_b = (df8.pipe(clean_column_names).rename(columns={"ia_investigator": "name"}).pipe(strip_titles).pipe(split_names_f_pf))
df8 = pd.concat([df8_a, df8_b])
df8[["middle_name"]] = ""


df9_a = df9.pipe(clean_column_names).reset_index()
df9_a = df9_a[["first_name", "last_name"]]
df9_b = df9.pipe(clean_column_names).rename(columns={"investigator": "name"}).pipe(strip_titles).pipe(split_names_f_pf)
df9_b = df9_b[["first_name", "last_name"]]
df9 = pd.concat([df9_a, df9_b])
df9[["middle_name"]] = ""

df10 = df10.pipe(clean_column_names).rename(columns={"officer_s": "name"}).pipe(strip_titles).pipe(split_names_v3)
df10["middle_name"] = ""

df11 = df11.pipe(strip_titles).pipe(split_names_v4)
df11["middle_name"] = ""

df12 = df12.pipe(strip_leading_aps).rename(columns={"value": "name"}).pipe(split_names)

df13 = df13.pipe(split_names_v4)

dfc = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13], axis=0)
dfc = dfc.reset_index()
dfc = dfc[["first_name", "middle_name", "last_name", "agency"]]
dfc = dfc.pipe(standardize_desc_cols, ["first_name", "middle_name", "last_name"])
dfc

In [630]:

def match_ok_names_to_personnel(ok, agency):
    dfa = ok[["first_name", "middle_name", "last_name", "notes", "city", "state"]]
    dfa["agency"] = ""
    dfa = dfa.pipe(gen_uid, ["first_name", "middle_name", "last_name"])
    dfa = dfa.drop_duplicates(subset=["uid"])
    dfa.loc[:, "fc"] = dfa.first_name.fillna("").map(lambda x: x[:1])

    dfb = agency[["first_name", "middle_name", "last_name", "agency"]]
    dfb = dfb.pipe(gen_uid, ["first_name", "middle_name", "last_name"])
    dfb = dfb.drop_duplicates(subset=["uid"])
    dfb["city"] = ""
    dfb["state"] = ""
    dfb["notes"] = ""
    dfb.loc[:, "fc"] = dfb.first_name.fillna("").map(lambda x: x[:1])

    matcher = ThresholdMatcher(
        ColumnsIndex([]),
        {
            "last_name": JaroWinklerSimilarity(),
            "first_name": JaroWinklerSimilarity(),
        },
        dfa,
        dfb,
    )
    decision = 0.800
    matcher.save_pairs_to_excel(
        ("../data/MA/oathkeepers_v_ma.xlsx"), decision
    )
    return matcher

In [631]:
match = match_ok_names_to_personnel(dfa, dfc)

C:\Users\PC\AppData\Local\Temp\ipykernel_11436\3161405069.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa["agency"] = ""
c:\Users\PC\Desktop\keepers\ok\notebooks\..\lib\uid.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, uid_name] = (


In [632]:
def read_matches():
    df = pd.read_excel("../data/MA/oathkeepers_v_ma.xlsx", sheet_name=1)
    return df 

In [633]:
matches = read_matches()

In [ ]:
matches.iloc[:50]